In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://es.wikipedia.org/wiki/Anexo:Pel%C3%ADculas_con_las_mayores_recaudaciones'
# Hacer la solicitud HTTP
response = requests.get(url)
response

<Response [200]>

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Anexo:Películas con las mayores recaudaciones - Wikipedia, la enciclopedia libre</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 

In [4]:
tabla = soup.find("table") 
tabla

<table class="wikitable sortable" style="text-align:center; margin=auto; font-size:100 %">
<caption>Películas con las mayores recaudaciones a nivel mundial (en <a href="/wiki/D%C3%B3lar_estadounidense" title="Dólar estadounidense">USD</a>)<sup class="reference separada" id="cite_ref-11"><a href="#cite_note-11"><span class="corchete-llamada">[</span>11<span class="corchete-llamada">]</span></a></sup>​
</caption>
<tbody><tr style="font-size:85 %">
<th>N.º
</th>
<th>Película
</th>
<th>Recaudación mundial
</th>
<th>Taquilla (en EE. UU.)
</th>
<th>Taquilla (fuera de EE. UU.)
</th>
<th>Presupuesto
</th>
<th>Distribuidora(s)
</th>
<th>Año de estreno
</th></tr>
<tr>
<th>1
</th>
<td><a href="/wiki/Avatar_(pel%C3%ADcula)" title="Avatar (película)">Avatar</a>
</td>
<td><span style="display:none">&amp;&amp;&amp;&amp;&amp;02923706026.&amp;&amp;&amp;&amp;&amp;0</span>2 923 706 026
</td>
<td><span style="display:none">&amp;&amp;&amp;&amp;&amp;&amp;0785221649.&amp;&amp;&amp;&amp;&amp;0</span>785 221 6

In [5]:
# Extraer las filas de la tabla
encabezado = tabla.find_all("th")
columnas = []
filas = tabla.find_all("tr")
encabezado = filas[0].find_all("th") 
columnas.append([e.text.strip().replace("\xa0","") for e in encabezado]) # 
columnas = columnas[0]

In [6]:
# Crear lista para almacenar los datos
data = []
# Recorrer cada fila
for fila in filas[1:]:
    celdas = fila.find_all(["td", "th"])  # Extraer celdas (th para encabezados)
    data.append([celda.text.strip().replace("\xa0","").split('&')[-1] for celda in celdas])  # Limpiar texto

# Crear DataFrame con los datos extraídos
df = pd.DataFrame(data, columns = columnas)

In [7]:
#df.columns = df.iloc[0]
df.head(3)

,N.º,Película,Recaudación mundial,Taquilla (en EE.UU.),Taquilla (fuera de EE.UU.),Presupuesto,Distribuidora(s),Año de estreno
0,1,Avatar,02923706026,"0785221649 (26,9 %)","02138484377 (73,1 %)",0246000000,20th Century Fox (Walt Disney Studios Motion P...,2009
1,2,Avengers: Endgame,02799439100,"0858373000 (30,7 %)","01941066100 (69,3 %)",0400000000,Walt Disney Studios Motion Pictures,2019
2,3,Avatar: El Camino del Agua,02320251738,"0684075767 (29,5 %)","01636174514 (70,5 %)",0356000000,Walt Disney Studios Motion Pictures,2022


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   N.º                         100 non-null    object
 1   Película                    100 non-null    object
 2   Recaudación mundial         100 non-null    object
 3   Taquilla (en EE.UU.)        100 non-null    object
 4   Taquilla (fuera de EE.UU.)  100 non-null    object
 5   Presupuesto                 100 non-null    object
 6   Distribuidora(s)            100 non-null    object
 7   Año de estreno              100 non-null    object
dtypes: object(8)
memory usage: 6.4+ KB


In [9]:
df['Taquilla_EEUU'] = df['Taquilla (en EE.UU.)'].str.split("(", expand = True)[0]
df['Taquilla_fuera_EEUU'] = df['Taquilla (fuera de EE.UU.)'].str.split("(", expand = True)[0]
df['pct_EEUU'] = df['Taquilla (en EE.UU.)'].str.split("(", expand = True)[1].str.replace(")","", regex = False)
df['pct_fuera_EEUU'] = df['Taquilla (fuera de EE.UU.)'].str.split("(", expand = True)[1].str.replace(")","", regex = False)
df.drop(['Taquilla (en EE.UU.)','Taquilla (fuera de EE.UU.)'], axis = 1, inplace = True)
df

,N.º,Película,Recaudación mundial,Presupuesto,Distribuidora(s),Año de estreno,Taquilla_EEUU,Taquilla_fuera_EEUU,pct_EEUU,pct_fuera_EEUU
0,1,Avatar,02923706026,0246000000,20th Century Fox (Walt Disney Studios Motion P...,2009,0785221649,02138484377,"26,9 %","73,1 %"
1,2,Avengers: Endgame,02799439100,0400000000,Walt Disney Studios Motion Pictures,2019,0858373000,01941066100,"30,7 %","69,3 %"
2,3,Avatar: El Camino del Agua,02320251738,0356000000,Walt Disney Studios Motion Pictures,2022,0684075767,01636174514,"29,5 %","70,5 %"
3,4,Titanic,02264743305,0200000000,20th Century Fox (Walt Disney Studios Motion P...,1997,0674292608,01590450697,"29,8 %","70,2 %"
4,5,Star Wars: Episodio VII - El despertar de la F...,02071310218,0306000000,Walt Disney Studios Motion Pictures,2015,0936662225,01134647993,"45,2 %","54,8 %"
...,...,...,...,...,...,...,...,...,...,...
95,96,Guardianes de la Galaxia vol. 3,0845555777,0250000000,Walt Disney Studios Motion Pictures,2023,0358995815,0486559962,"42,5 %","57,5 %"
96,97,El Origen,0839030630,0160000000,Warner Bros.,2010,0292587330,0544260772,33.6 %,66.4 %
97,98,Transformers: la venganza de los caídos,0836303693,0200000000,Paramount Pictures,2009,0402111870,0434191823,"48,2 %","51,7 %"
98,99,Spider-Man,0825025036,0140000000,Sony Pictures/Walt Disney Studios Motion Pictu...,2002,0407022860,0418002176,"49,3 %","50,7 %"


In [10]:
col_int = ['Recaudación mundial', 'Presupuesto','Taquilla_EEUU','Taquilla_fuera_EEUU']

for c in col_int:
    print(c)
    df.loc[df[c]=='-', c] = '0'
    df[c] = df[c].astype(float)

df.info()

Recaudación mundial


ValueError: could not convert string to float: '0Expresión errónea: número inesperado.Expresión errónea: número inesperado1 720 000 000'

In [135]:
df['Pres'] = df['Presupuesto'] + 1

TypeError: can only concatenate str (not "int") to str

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   N.º                  100 non-null    object
 1   Película             100 non-null    object
 2   Recaudación mundial  100 non-null    object
 3   Presupuesto          100 non-null    object
 4   Distribuidora(s)     100 non-null    object
 5   Año de estreno       100 non-null    object
 6   Taquilla_EEUU        100 non-null    object
 7   Taquilla_fuera_EEUU  100 non-null    object
 8   pct_EEUU             99 non-null     object
 9   pct_fuera_EEUU       100 non-null    object
dtypes: object(10)
memory usage: 7.9+ KB
